In [ ]:
!pip install "paho-mqtt<2.0.0"

In [ ]:
import paho.mqtt.client as mqtt
import random
import time
import threading

broker_address = "broker.emqx.io"
port = 1883
username = "test"
password = "1"

# 定义发布心跳数据的函数
def publish_heartbeat(client_name, topic):
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print(f"{client_name} connected to broker")
        else:
            print(f"{client_name} connection failed with code {rc}")

    client = mqtt.Client(client_name)
    client.username_pw_set(username, password)
    client.on_connect = on_connect

    client.connect(broker_address, port)
    client.loop_forever()

    while True:
        heartbeat = random.randint(60, 130)
        client.publish(topic, heartbeat)
        print(f"{client_name} sent heartbeat: {heartbeat}")
        time.sleep(5)

# 创建多个线程，每个线程负责一个客户端
threads = []

client_info = [
    {"client_name": "client1", "topic": "patient/num1/heartbeat"},
    {"client_name": "client2", "topic": "patient/num2/heartbeat"},
    {"client_name": "client3", "topic": "patient/num3/heartbeat"},
]

for info in client_info:
    thread = threading.Thread(target=publish_heartbeat, args=(info["client_name"], info["topic"]))
    threads.append(thread)
    thread.start()

# 保持主线程运行
for thread in threads:
    thread.join()
